In [1]:
from bbq.decoder import d_osd, bp_osd, belief_propagation, _syn_inv_permute_field
from bbq.utils import det_to_err, err_to_det, find_pivots, rref
import numpy as np
from ldpc import BpDecoder, bposd_decoder
from bbq.polynomial import Polynomial
from bbq.bbq_code import BivariateBicycle
import matplotlib.pyplot as plt
import json
import datetime
import galois

In [2]:
def simulate(field, h, l, max_iter, num_failures, physical_error):

    results = []

    for p in physical_error:

        failures = 0
        num_trials = 0

        while failures < num_failures:
            # Generate syndrome
            n_qudits = h.shape[1]
            error = np.zeros(n_qudits, dtype=int)
            error_mask = np.random.rand(n_qudits) < p
            for i in np.where(error_mask)[0]:
                error[i] = np.random.randint(1, field)
            syndrome = (h @ error) % field

            # Construct error probability
            channel_prob_x = np.ones(n_qudits) * p

            x_prior = np.zeros((len(channel_prob_x), field), dtype=float)

            for i, prob in enumerate(channel_prob_x):
                x_prior[i, 0] = 1 - prob
                for j in range(1, field):
                    x_prior[i, j] = prob / (field - 1)

            # Decode
            # guessed_error, decoder_success, bp_success, posterior = belief_propagation(field, h, syndrome, x_prior, max_iter, debug=True)
            guessed_error, decoder_success, bp_success, posterior = bp_osd(field, h, syndrome, x_prior, max_iter, order=0, debug=True)
            error_difference = (error - guessed_error) % field
            logical_effect = (np.array(l) @ error_difference) % field

            # Check success
            # if np.any(logical_effect != 0) or not decoder_success:
            if np.any(logical_effect != 0):
                failures += 1
                print(f'Found {failures} / {num_failures}, with num_trials : {num_trials}')
            elif num_trials % 100 == 0:
                print(f'UPDATE: Found {failures} / {num_failures}, with num_trials : {num_trials}')

            num_trials += 1

        results.append(num_trials)

        print(f'Finished p={p} with num_trials={num_trials}')
        print('------------------------------------------------------------------')
    return results

In [3]:
a = Polynomial(2, np.array([[1, 0], [1, 0]]))
b = Polynomial(2, np.array([[1, 1], [0, 0]]))
bb2 = BivariateBicycle(a, b, 3, 3, 1)
h2 = bb2.hx
l2 = bb2.x_logicals
bb25 = BivariateBicycle(a, b, 5, 5, 1)
h25 = bb25.hx
l25 = bb25.x_logicals
bb27 = BivariateBicycle(a, b, 7, 7, 1)
h27 = bb27.hx
l27 = bb27.x_logicals

a = Polynomial(3, np.array([[1, 0], [-1, 0]]))
b = Polynomial(3, np.array([[1, -1], [0, 0]]))
bb3 = BivariateBicycle(a, b, 3, 3, 1)
h3 = bb3.hx
l3 = bb3.x_logicals
bb35 = BivariateBicycle(a, b, 5, 5, 1)
h35 = bb35.hx
l35 = bb35.x_logicals
bb37 = BivariateBicycle(a, b, 7, 7, 1)
h37 = bb37.hx
l37 = bb37.x_logicals

a = Polynomial(5, np.array([[1, 0], [-1, 0]]))
b = Polynomial(5, np.array([[1, -1], [0, 0]]))
bb5 = BivariateBicycle(a, b, 3, 3, 1)
h5 = bb5.hx
l5 = bb5.x_logicals
bb55 = BivariateBicycle(a, b, 5, 5, 1)
h55 = bb55.hx
l55 = bb55.x_logicals
bb57 = BivariateBicycle(a, b, 7, 7, 1)
h57 = bb57.hx
l57 = bb57.x_logicals

max_iter = 1000
num_failures = 3
long_physical_error = np.logspace(-2.7, -1, 10)
physical_error = np.flip(np.logspace(-1.7, -0.7, 10))
ext_physical_error = np.flip(np.logspace(-1, -0.8, 10))

In [ ]:
results = np.flip(simulate(5, h5, l5, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h5, l5, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

In [ ]:
results